In [1]:
import os
import pandas as pd

In [2]:
from rapidfuzz import process as proc
from rapidfuzz import fuzz

In [3]:
from tqdm import tqdm

In [4]:
from fastparquet import ParquetFile, write as parq_write

In [5]:
# Load methods for text processing
from src.utils import (
    simple_process_item, count_common_digits, count_common_words, count_digit_share,
    remove_numbers, get_product_group, get_parent_group, get_excluded_list
)

[nltk_data] Downloading package punkt to /home/varsey/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/varsey/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_ru to
[nltk_data]     /home/varsey/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_ru is already up-to-
[nltk_data]       date!


## Load dataset and preprocess items names (lowercase and stopwords removal)

In [6]:
df_incoming = pd.read_excel(f'{os.getcwd()}/data/drl-data.xlsx', sheet_name='Inventory Out')
df_outcoming = pd.read_excel(f'{os.getcwd()}/data/drl-data.xlsx', sheet_name='Inventory In')

In [7]:
df_outcoming['unit'] = df_incoming['Unit'].str.lower()
df_incoming['unit'] = df_incoming['Unit'].str.lower()

In [8]:
exclude = get_excluded_list()

In [9]:
df_outcoming['items'] = df_outcoming['Product or Service Name'].apply(lambda x: simple_process_item(x, exclude))
df_incoming['items'] = df_incoming['Product or Service Name'].apply(lambda x: simple_process_item(x, exclude))

## Create lists of pairs for each item in df_incoming df with scores for each pair

### The cell below take up to 4hs to run. You can use preprocessed data below

In [10]:
%%time

to_find = df_incoming['items'].to_list()
candidates = set(df_outcoming['items'].to_list())

outcoming, incoming1, incoming2, score1, score2, score_w, score_d = [], [], [], [], [], [], []
for item in tqdm(to_find[:]):
    res1 = proc.extract(
                item,
                candidates,
                scorer=fuzz.partial_token_sort_ratio,
                limit=1
    )[0]
    res2 = proc.extract(
                item,
                candidates,
                scorer=fuzz.partial_ratio,
                limit=1
    )[0]
    outcoming.append(item)
    incoming1.append(res1[0])
    incoming2.append(res2[0])
    score1.append(res1[1])
    score2.append(res2[1])
    if len(res1) > 0:
        score_w.append(count_common_words(item, res1[0]))
        score_d.append(count_common_digits(item, res1[0]) / count_digit_share(res1[0]))


100%|██████████| 937415/937415 [3:50:17<00:00, 67.84it/s]   

CPU times: user 3h 48min 44s, sys: 34.9 s, total: 3h 49min 18s
Wall time: 3h 50min 17s


## Create df with pairs of matched items and corresponing scores

In [11]:
comp = pd.DataFrame()
comp['right'] = outcoming
comp['left1'] = incoming1
comp['left2'] = incoming2

comp['score1'] = score1
comp['score2'] = score2
# Harmonical score for quality decision
comp['score3'] = 2 * comp['score1'] *comp['score2'] /(comp['score1'] + comp['score2'] )

comp['score_w'] = score_w
comp['score_d'] = score_d
# Metric for numbers in items name comparison
comp['score_d'] = comp['score_d'] /10

### Write preprocessed data to file to save time

In [12]:
parq_write(
    f'{os.getcwd()}/comp_full.parq',
    comp,
    compression='GZIP'
)

### Load preprocessed data from file

In [13]:
comp_parq = ParquetFile(f'{os.getcwd()}/comp_full.parq',)
comp = comp_parq.to_pandas()

### Concat incoming items with mathed pairs df

In [14]:
sub_result = pd.concat(
    [
        df_incoming,
        comp
    ],
    axis=1,
)
sub_result.shape

(937415, 14)

In [15]:
sub_result.sample(20)

,#,Product or Service Name,Unit,Quantity/Volume,unit,items,right,left1,left2,score1,score2,score3,score_w,score_d
411594,411595,ULKER SUD ALPELLA SOKOLADLI 180 ML 1X27,EDED,12,eded,ulker sud alpella sokoladli 180 ml 1 x 27,ulker sud alpella sokoladli 180 ml 1 x 27,süd alpella şokoladlı 1.5 pct 180 ml x 27,süd alpella şokoladlı 1.5 pct 180 ml x 27,81.159420,78.947368,80.038113,0.400000,0.377143
599780,599781,PINAR SUT KIDO CILEKLI 180 ML 1x27,ADET.ML,15,adet.ml,pinar sut kido cilekli 180 ml 1 x 27,pinar sut kido cilekli 180 ml 1 x 27,kif,kif,66.666667,66.666667,66.666667,0.000000,0.000000
612572,612573,DORITOS ACILI 130 GR 1 x 16,EDED,16,eded,doritos acili 130 gr 1 x 16,doritos acili 130 gr 1 x 16,doritos acılı 130 qr x 16,doritos acılı 130 qr x 16,88.000000,80.851064,84.274194,0.500000,0.320000
900803,900804,SEVIMLI DAD GATIG 0.5% 8 KG 1X1,EDED,6,eded,sevimli dad gatig 0.5 pct 8 kg 1 x 1,sevimli dad gatig 0.5 pct 8 kg 1 x 1,sevimli dad gatig 4.5 pct 900 gr 1 x 6,sevimli dad gatig 4.5 pct 900 gr 1 x 6,84.507042,86.111111,85.301536,1.000000,0.165714
360979,360980,QLAVPRODUKT ADJIKA PIKATNAYA 170 QR 1X15,EDED,5,eded,qlavprodukt adjika pikatnaya 170 qr 1 x 15,qlavprodukt adjika pikatnaya 170 qr 1 x 15,k 45 19,fan,66.666667,66.666667,66.666667,0.000000,0.000000
548642,548643,LAYS STIX KETCHUP 125 GR 1X14,EDED,2,eded,lays stix ketchup 125 gr 1 x 14,lays stix ketchup 125 gr 1 x 14,lays stiks ketçup 125 q dsp x 14,lays stiks ketçup 125 q dsp x 14,74.576271,77.419355,75.971223,0.333333,0.400000
332695,332696,MILLA PENDIR BEYAZ 250 GR 1X24,EDED,4,eded,milla pendir beyaz 250 gr 1 x 24,milla pendir beyaz 250 gr 1 x 24,milla pendir bəyaz 250 qr x 24,milla pendir bəyaz 250 qr x 24,86.666667,86.666667,86.666667,0.666667,0.384000
504682,504683,SEVIMLI DAD GATIG SUSE 0.5% 2 KG 1X5,EDED,5,eded,sevimli dad gatig suse 0.5 pct 2 kg 1 x 5,sevimli dad gatig suse 0.5 pct 2 kg 1 x 5,sevimli dad gatig 3 pct 450 gr 1 x 22,sevimli dad gatig 1.5 pct 450 gr 1 x 22,83.783784,79.487179,81.578947,0.800000,0.114286
323181,323182,BERRAK SALATALIK NO2 BULGAR USULLU 720 ML 1X12,ADET.ML,12,adet.ml,berrak salatalik 2 bulgar usullu 720 ml 1 x 12,berrak salatalik 2 bulgar usullu 720 ml 1 x 12,plastik qutu,7 up 1 lt x 12,66.666667,66.666667,66.666667,0.000000,0.000000
629127,629128,KOMILI AYCICEK YAGI 1 LT PET 1X12,ADET.LT,12,adet.lt,komili aycicek yagi 1 lt 1 x 12,komili aycicek yagi 1 lt 1 x 12,yağ,7 up 1 lt x 12,66.666667,76.923077,71.428571,0.000000,0.000000


### Merge outcoming data to preprocessed pairs of matched products

In [16]:
compilation_full = sub_result.merge(df_outcoming, how='left', left_on='left1', right_on='items')
compilation_full = compilation_full.drop_duplicates(subset=['#_x']) 
compilation_full.shape

(937415, 20)

In [17]:
columns_to_use = [
          'items_x', 'items_y',
          'left1', 'left2',
          'Quantity/Volume_x', 'unit_x',
          'Quantity/Volume_y', 'unit_y',
          'score1', 'score2', 'score3', 'score_w', 'score_d',
          'Product or Service Name_x', 'Product or Service Name_y'
]

In [18]:
compilation_full = compilation_full[columns_to_use]

In [19]:
df_outcoming.shape, df_incoming.shape, compilation_full.shape

((16947, 6), (937415, 6), (937415, 15))

### Filter data by scores metrics to robust result

In [20]:
comp_true = compilation_full[
    (compilation_full.score3 > 78) 
    & 
    (compilation_full.score_d > 0.17)
]

### Exclude from pairs data we doubt according to the scores

In [21]:
comp_doubt = compilation_full.drop(comp_true.index, axis=0)


In [22]:
compilation_full.shape, comp_true.shape

((937415, 15), (266683, 15))

### Share of data we consider is matched correctly

In [23]:
comp_true.shape[0] / compilation_full.shape[0]

0.2844876602145261

In [24]:
compilation_scored = comp_true

### Preprocess units to see in the result table if they are the same for left and right item in matched pair

In [25]:
compilation_scored['unit_common'] = compilation_scored['unit_x'] + ' ' +  compilation_scored['unit_y']
compilation_scored['unit_common_set'] = compilation_scored['unit_common'].apply(lambda x: set(str(x).replace('.', ' ').split()))

/tmp/ipykernel_42468/3152849601.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  compilation_scored['unit_common'] = compilation_scored['unit_x'] + ' ' +  compilation_scored['unit_y']
/tmp/ipykernel_42468/3152849601.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  compilation_scored['unit_common_set'] = compilation_scored['unit_common'].apply(lambda x: set(str(x).replace('.', ' ').split()))


In [26]:
compilation_scored['unit_common_str'] = compilation_scored['unit_common_set'].apply(
    lambda x: str(list(x)[0]) if len(list(x)) == 0 else ' '.join(list(x))
)

/tmp/ipykernel_42468/3820387171.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  compilation_scored['unit_common_str'] = compilation_scored['unit_common_set'].apply(


### Convert quantaties to float for aggregating later

In [27]:
compilation_scored['quant_in'] = compilation_scored['Quantity/Volume_x'].astype('str').str.replace(',', '.').fillna(0).astype('float')
compilation_scored['quant_out'] = compilation_scored['Quantity/Volume_y'].astype('str').str.replace(',', '.').fillna(0).astype('float')

/tmp/ipykernel_42468/2514422162.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  compilation_scored['quant_in'] = compilation_scored['Quantity/Volume_x'].astype('str').str.replace(',', '.').fillna(0).astype('float')
/tmp/ipykernel_42468/2514422162.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  compilation_scored['quant_out'] = compilation_scored['Quantity/Volume_y'].astype('str').str.replace(',', '.').fillna(0).astype('float')


In [28]:
compilation_scored.shape

(266683, 20)

In [42]:
# x - incoming y - outcoming
compilation_scored[
    [
        'items_x', 'items_y',
        'score3', 'score1', 'score2', 'score_w', 'score_d',
        'Product or Service Name_x', 'Product or Service Name_y'
    ]
].sample(250)

,items_x,items_y,score3,score1,score2,score_w,score_d,Product or Service Name_x,Product or Service Name_y
7722561,sevimli dad kefir 2.5 pct 900 gr 1 x 12,sevimli dad kefir 2.5 pct 900 gr,100.000000,100.000000,100.000000,1.000000,0.260000,SEVIMLI DAD KEFIR 2.5% 900 GR 1x12,SEVIMLI DAD KEFIR 2.5% 900 GR
12427599,icimino sud sokoladli 180 ml 1 x 27,süd i̇çi̇mino şokoladlı 180 ml x 27,79.365079,79.365079,79.365079,0.000000,0.464000,ICIMINO SUD SOKOLADLI 180 ML 1X27,"Süd ""İÇİMINO"" Şokoladlı 180 ML x 27"
2693525,sevimli dad gatig 3 pct 450 gr 1 x 22,sevimli dad gatig 3 pct 450 gr 1 x 22,100.000000,100.000000,100.000000,1.000000,0.400000,SEVIMLI DAD GATIG 3% 450 GR 1X22,SEVIMLI DAD GATIG 3% 450 GR 1X22
6709512,ulker sud alpella sokoladli 180 ml 1 x 27,süd alpella şokoladlı 1.5 pct 180 ml x 27,80.038113,81.159420,78.947368,0.400000,0.377143,ULKER SUD ALPELLA SOKOLADLI 180 ML 1X27,"Süd ""Alpella"" Şokoladlı 1.5% 180 ML x 27"
3180664,sochnaya dolina multivitamin 0.95 lt 1 x 12,multivitamin 1 lt paket,83.410565,78.260870,89.285714,0.333333,1.333333,SOCHNAYA DOLINA MULTIVITAMIN 0.95 LT 1x12,Multivitamin 1 lt paket
...,...,...,...,...,...,...,...,...,...
12409976,milla ayran naneli 200 ml 1 x 12,milla ayran nanəli 200 ml x 12,93.868659,96.666667,91.228070,0.666667,0.384000,MILLA AYRAN NANELI 200ML 1X12,Milla Ayran Nanəli 200ml *12
13359811,lays paprika 90 gr 1 x 25 p,lays paprika 90 q dsp x 25,84.268126,82.352941,86.274510,1.000000,0.400000,LAYS PAPRIKA 90 GR 1 x 25 P,Lay's Paprika 90q DSP X25
5678092,cheetos biftekli kitir 50 gr 1 x 24,cheetos biftekli 50 qr x 24,84.241824,92.592593,77.272727,0.800000,0.440000,CHEETOS BIFTEKLI KITIR 50 GR 1X24,Cheetos Biftekli 50 qr X24
1551448,sevimli dad dovga 2.5 pct 900 gr 1 x 12,sevimli dad gatig 4.5 pct 900 gr 1 x 6,86.646154,85.333333,88.000000,0.500000,0.276190,SEVIMLI DAD DOVGA 2.5% 900 GR 1x12,SEVIMLI DAD GATIG 4.5% 900 GR 1X6


### Get groups from preprocessed items names

In [30]:
compilation_scored['items_x_numberless'] = compilation_scored['items_x'].apply(lambda x: remove_numbers(x))

/tmp/ipykernel_42468/3752593017.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  compilation_scored['items_x_numberless'] = compilation_scored['items_x'].apply(lambda x: remove_numbers(x))


In [31]:
compilation_scored['product_group'] = compilation_scored['items_x_numberless'].apply(lambda x: get_product_group(x))
compilation_scored['parent_group'] = compilation_scored['product_group'].apply(lambda x: get_parent_group(x))

/tmp/ipykernel_42468/3845966172.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  compilation_scored['product_group'] = compilation_scored['items_x_numberless'].apply(lambda x: get_product_group(x))
/tmp/ipykernel_42468/3845966172.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  compilation_scored['parent_group'] = compilation_scored['product_group'].apply(lambda x: get_parent_group(x))


### Aggregated result by items and their groups

In [55]:
general_aggregate = compilation_scored.groupby(
    by=['parent_group', 'product_group', 'Product or Service Name_x', 'Product or Service Name_y', 'unit_common_str']
).agg(
    {'quant_in':'sum','quant_out':'sum'}
).sort_values(
    ['unit_common_str']
)

general_aggregate

quant_in  \
parent_group product_group   Product or Service Name_x          Product or Service Name_y              unit_common_str             
omnik        omnik           Omnik N30                          NOVALEX N30                            adet baql cl         20.0   
kefir        president kefir PRESIDENT KEFIR 2,5% 450 GR 1X8    Xama "President" smetana 15% 350gr x 8 adet eded cl       3661.0   
xama         president xama  PRESIDENT XAMA 30%, 200 GR 1X8     Xama "President" smetana 15% 350gr x 8 adet eded cl         73.0   
sud          sud icim        SUD ICIM YARIM YAGLI 1 LT 1X12     Süd "İÇİM" Yarim Yagli 1 LT x 12       adet eded cl        518.0   
xama         president xama  PRESIDENT XAMA 15% 350 GR 1X8      Xama "President" smetana 15% 350gr x 8 adet eded cl       6184.0   
...                                                                                                                          ...   
ekzoderil    ekzoderil       Ekzoderil 1% 20ml N1 03.2025 (600) EXODERIL 1% 20ML SOL OTC AZ            ədəd ml adet         36.0   
nok          nok             5-nok 50mg N50 10.2024             5-NOK 50 MG N 50                       ədəd qram            10.0   
                             5-nok 50mg N50 02.2025 (200)       5-NOK 50 MG N 50                       ədəd qram            20.0   
nakom        nakom           Nakom 250+25mg №100 09.2022        NAKOM 250 MG+25 MG 100 TAB             ədəd qram             3.0   
esomeprazol  esomeprazol     Esomeprazol 20mg N28 08.2021       ESOMEPRAZOL 20 MG N 28                 ədəd qram             1.0   

                                                                                                                        quant_out  
parent_group product_group   Product or Service Name_x          Product or Service Name_y              unit_common_str             
omnik        omnik           Omnik N30                          NOVALEX N30                            adet baql cl           1.0  
kefir        president kefir PRESIDENT KEFIR 2,5% 450 GR 1X8    Xama "President" smetana 15% 350gr x 8 adet eded cl      316528.0  
xama         president xama  PRESIDENT XAMA 30%, 200 GR 1X8     Xama "President" smetana 15% 350gr x 8 adet eded cl        8176.0  
sud          sud icim        SUD ICIM YARIM YAGLI 1 LT 1X12     Süd "İÇİM" Yarim Yagli 1 LT x 12       adet eded cl        9030.0  
xama         president xama  PRESIDENT XAMA 15% 350 GR 1X8      Xama "President" smetana 15% 350gr x 8 adet eded cl      435080.0  
...                                                                                                                           ...  
ekzoderil    ekzoderil       Ekzoderil 1% 20ml N1 03.2025 (600) EXODERIL 1% 20ML SOL OTC AZ            ədəd ml adet        4000.0  
nok          nok             5-nok 50mg N50 10.2024             5-NOK 50 MG N 50                       ədəd qram           8000.0  
                             5-nok 50mg N50 02.2025 (200)       5-NOK 50 MG N 50                       ədəd qram           4000.0  
nakom        nakom           Nakom 250+25mg №100 09.2022        NAKOM 250 MG+25 MG 100 TAB             ədəd qram            720.0  
esomeprazol  esomeprazol     Esomeprazol 20mg N28 08.2021       ESOMEPRAZOL 20 MG N 28                 ədəd qram            100.0  

[596 rows x 2 columns]

### Final table with parent groups (brand) invetory balances

In [49]:
brand_agg = compilation_scored.groupby(
    ['parent_group', 'product_group', 'Product or Service Name_x', 'unit_common_str']
).agg(
    {'quant_in': 'sum', 'quant_out': 'sum'}
).reset_index()

brand_agg.rename(columns={'quant_in': 'brand_inventory_in', 'quant_out': 'brand_inventory_out'}, inplace=True)

result = pd.merge(brand_agg, general_aggregate, on='parent_group', how='left')

In [50]:
result.groupby(
    by=['parent_group', 'product_group', 'Product or Service Name_x', 'unit_common_str']
).agg(
    {'quant_in':'sum','quant_out':'sum', 'brand_inventory_in': 'max', 'brand_inventory_out': 'max'}
)

quant_in  \
parent_group product_group    Product or Service Name_x           unit_common_str             
                              7UP 0.5 LT 1X12                     adet lt eded       3808.0   
                              7UP 1 LT 1X12                       adet lt eded       3808.0   
                              7UP 2 LT 1X6                        adet lt eded       3808.0   
                              PH-1-10                             qutu eded          3808.0   
acc          acc              ACC 200mg N20 07.2021 (300)         eded                311.0   
...                                                                                     ...   
xeste        xeste alti       XESTE ALTI BEZ DOUBLE ACT 60X90 N30 ədəd eded            41.0   
xrustem      xrustem biftekli XRUSTEM BIFTEKLI 75 GR              ədəd kg adet       2256.0   
yod          yod              Yod 5%                              fl eded             200.0   
                              yod 5%                              flak eded           200.0   
                                                                  ədəd eded           200.0   

                                                                                   quant_out  \
parent_group product_group    Product or Service Name_x           unit_common_str              
                              7UP 0.5 LT 1X12                     adet lt eded      157740.0   
                              7UP 1 LT 1X12                       adet lt eded      157740.0   
                              7UP 2 LT 1X6                        adet lt eded      157740.0   
                              PH-1-10                             qutu eded         157740.0   
acc          acc              ACC 200mg N20 07.2021 (300)         eded               17860.0   
...                                                                                      ...   
xeste        xeste alti       XESTE ALTI BEZ DOUBLE ACT 60X90 N30 ədəd eded             10.0   
xrustem      xrustem biftekli XRUSTEM BIFTEKLI 75 GR              ədəd kg adet      808704.0   
yod          yod              Yod 5%                              fl eded              600.0   
                              yod 5%                              flak eded            600.0   
                                                                  ədəd eded            600.0   

                                                                                   brand_inventory_in  \
parent_group product_group    Product or Service Name_x           unit_common_str                       
                              7UP 0.5 LT 1X12                     adet lt eded                 1142.0   
                              7UP 1 LT 1X12                       adet lt eded                 1214.0   
                              7UP 2 LT 1X6                        adet lt eded                 1448.0   
                              PH-1-10                             qutu eded                       4.0   
acc          acc              ACC 200mg N20 07.2021 (300)         eded                            1.0   
...                                                                                               ...   
xeste        xeste alti       XESTE ALTI BEZ DOUBLE ACT 60X90 N30 ədəd eded                      41.0   
xrustem      xrustem biftekli XRUSTEM BIFTEKLI 75 GR              ədəd kg adet                 2256.0   
yod          yod              Yod 5%                              fl eded                        80.0   
                              yod 5%                              flak eded                      90.0   
                                                                  ədəd eded                      30.0   

                                                                                   brand_inventory_out  
parent_group product_group    Product or Service Name_x           unit_common_str                       
                      

In [46]:
general_aggregate.to_csv('general_aggregate.csv')

In [54]:
result.rename(
    columns={
        'parent_group': 'Parent Group Name',
        'product_group': 'Product Group',
        'brand_inventory_in': 'Group Inventory In Quantity',
        'brand_inventory_out': 'Group Inventory Out Quantity',
        'Product or Service Name_x': 'Individual Items',
        'unit_common_str': 'Product Unit',
        'quant_in': 'Inventory In Quantity',
        'quant_out': 'Inventory Out Quantity',
    },
).groupby(
    by=['Parent Group Name', 'Product Group', 'Individual Items', 'Product Unit',]
).agg(
    {'Inventory In Quantity':'sum', 'Inventory Out Quantity' :'sum', 'Group Inventory In Quantity': 'max', 'Group Inventory Out Quantity': 'max'}
).to_csv('result.csv')

In [53]:
result

,Parent Group Name,Product Group,Individual Items,Product Unit,Group Inventory In Quantity,Group Inventory Out Quantity,Inventory In Quantity,Inventory Out Quantity
0,,,7UP 0.5 LT 1X12,adet lt eded,1142.0,59400.0,1142.0,59400.0
1,,,7UP 0.5 LT 1X12,adet lt eded,1142.0,59400.0,1214.0,70200.0
2,,,7UP 0.5 LT 1X12,adet lt eded,1142.0,59400.0,1448.0,27840.0
3,,,7UP 0.5 LT 1X12,adet lt eded,1142.0,59400.0,4.0,300.0
4,,,7UP 1 LT 1X12,adet lt eded,1214.0,70200.0,1142.0,59400.0
...,...,...,...,...,...,...,...,...
7601,yod,yod,yod 5%,flak eded,90.0,300.0,90.0,300.0
7602,yod,yod,yod 5%,flak eded,90.0,300.0,30.0,100.0
7603,yod,yod,yod 5%,ədəd eded,30.0,100.0,80.0,200.0
7604,yod,yod,yod 5%,ədəd eded,30.0,100.0,90.0,300.0


In [59]:
len(compilation_scored['Product or Service Name_x'].unique())

554

In [60]:
len(compilation_scored['Product or Service Name_y'].unique())

361

In [61]:
len(df_incoming['Product or Service Name'].unique())

5125

In [63]:
len(df_outcoming['Product or Service Name'].unique())

2985

In [64]:
len(compilation_scored['Product or Service Name_x'].unique()) / len(compilation_scored['Product or Service Name_y'].unique()), len(df_incoming['Product or Service Name'].unique()) / len(df_outcoming['Product or Service Name'].unique())

(1.5346260387811634, 1.7169179229480738)